In [1]:
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"

!pip install scikit-learn
!pip install -qqq --upgrade unsloth vllm
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.2/438.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 148.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
import re

# ============================================================================
# STEP 1: Load Model
# ============================================================================
max_seq_length = 512  # Reduced since no reasoning needed
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-10 07:10:35 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 10-10 07:10:44 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker.model_runner'
==((====))==  Unsloth 2025.10.1: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.11.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 88.97%
Unsloth: Your GPU has CUDA compute capability 8.0 w

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.10.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:
# ============================================================================
# STEP 2: Simplified System Prompt (NO REASONING)
# ============================================================================
system_prompt = """You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, predict if there will be a failure or if operation is normal.
Respond with ONLY one word: either 'failure' or 'normal'."""

# ============================================================================
# STEP 3: Create Simple Chat Template
# ============================================================================
# Properly escape the system prompt for Jinja2
escaped_system_prompt = system_prompt.replace("'", "\\'")

chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        f"{{{{ '{escaped_system_prompt}' + eos_token }}}}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] + '\nAnswer: ' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"
tokenizer.chat_template = chat_template

# ============================================================================
# STEP 4: Load and Prepare Machine Sensor Dataset
# ============================================================================
csv_file = 'data.csv'
print("Path to dataset files:", csv_file)

machine_df = pd.read_csv(csv_file)
machine_df.columns = machine_df.columns.str.strip().str.lower()

# Map fail column to descriptive names
machine_df['status'] = machine_df['fail'].map({
    0: 'normal',
    1: 'failure'
})

print(f"Machine dataset shape: {machine_df.shape}")
print(f"\nClass distribution:")
print(machine_df['status'].value_counts())
print(f"\nFirst few rows:")
print(machine_df.head())

# Handle missing values
machine_df = machine_df.dropna()

Path to dataset files: data.csv
Machine dataset shape: (944, 11)

Class distribution:
status
normal     551
failure    393
Name: count, dtype: int64

First few rows:
   footfall  tempmode  aq  uss  cs  voc  rp  ip  temperature  fail   status
0         0         7   7    1   6    6  36   3            1     1  failure
1       190         1   3    3   5    1  20   4            1     0   normal
2        31         7   2    2   6    1  24   6            1     0   normal
3        83         4   3    4   5    1  28   6            1     0   normal
4       640         7   5    6   4    0  68   6            1     0   normal


In [8]:
# ============================================================================
# STEP 5: Create Simple Training Examples (NO REASONING)
# ============================================================================
def create_simple_example(row):
    """Create a simple direct answer without reasoning"""
    return row['status']

# Create balanced pre-training dataset
n_samples_per_class = 20  # Increased for better initial learning
pretrain_df = pd.concat([
    machine_df[machine_df['status'] == 'normal'].sample(n=min(n_samples_per_class, len(machine_df[machine_df['status'] == 'normal'])), random_state=42),
    machine_df[machine_df['status'] == 'failure'].sample(n=min(n_samples_per_class, len(machine_df[machine_df['status'] == 'failure'])), random_state=42)
]).reset_index(drop=True)

print(f"\nPre-training dataset class distribution:")
print(pretrain_df['status'].value_counts())

def format_pretrain_dataset(row):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""

    return {
        "Messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
            {"role": "assistant", "content": row['status']},
        ]
    }

pretrain_dataset = pretrain_df.apply(format_pretrain_dataset, axis=1).tolist()
pretrain_dataset = pd.DataFrame(pretrain_dataset)

# Tokenize
pretrain_dataset["text"] = tokenizer.apply_chat_template(
    pretrain_dataset["Messages"].values.tolist(),
    tokenize=False
)
pretrain_dataset = Dataset.from_pandas(pretrain_dataset)

print(f"\nPre-training dataset size: {len(pretrain_dataset)}")
print("\nExample formatted message:")
print(tokenizer.apply_chat_template(pretrain_dataset[0]["Messages"], tokenize=False))


Pre-training dataset class distribution:
status
normal     20
failure    20
Name: count, dtype: int64

Pre-training dataset size: 40

Example formatted message:
You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, predict if there will be a failure or if operation is normal.
Respond with ONLY one word: either 'failure' or 'normal'.<|endoftext|>Predict machine failure based on sensor data:
Footfall: 0
Temperature Mode: 0
Air Quality: 3 AQI
Ultrasonic Sensor: 4
Current Sensor: 4 A
VOC Level: 0 ppb
Rotational Position: 48 RPM
Input Pressure: 6 PSI
Temperature: 24°C
Answer: normal<|endoftext|>


In [9]:
# ============================================================================
# STEP 6: Pre-Fine-Tune for Simple Format
# ============================================================================
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=pretrain_dataset,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=4,  # Increased batch size since no reasoning
        gradient_accumulation_steps=1,
        warmup_steps=5,
        num_train_epochs=5,  # More epochs for better learning
        learning_rate=2e-4,
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs_no_reasoning",
        report_to="none",
    ),
)

print("\n" + "="*50)
print("Starting pre-training (no reasoning)...")
print("="*50)
trainer.train()

# Test the model
text = tokenizer.apply_chat_template(
    pretrain_dataset[0]["Messages"][:2],
    tokenize=False,
    add_generation_prompt=True,
)

from transformers import TextStreamer
print("\n" + "="*50)
print("Testing pre-trained model (no reasoning):")
print("="*50)
_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    temperature=0,
    max_new_tokens=50,  # Much shorter since just one word expected
    streamer=TextStreamer(tokenizer, skip_prompt=False),
)

# Clean up
del pretrain_dataset
torch.cuda.empty_cache()
import gc
gc.collect()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/40 [00:00<?, ? examples/s]


Starting pre-training (no reasoning)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 40 | Num Epochs = 5 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
5,2.316500
10,0.633600
15,0.237700
20,0.195900
25,0.186000
30,0.186400
35,0.176600
40,0.176100
45,0.168600
50,0.168400


Unsloth: Will smartly offload gradients to save VRAM!

Testing pre-trained model (no reasoning):
You are an industrial maintenance expert analyzing machine sensor data.
Given sensor readings from a machine, predict if there will be a failure or if operation is normal.
Respond with ONLY one word: either 'failure' or 'normal'.<|endoftext|>Predict machine failure based on sensor data:
Footfall: 0
Temperature Mode: 0
Air Quality: 3 AQI
Ultrasonic Sensor: 4
Current Sensor: 4 A
VOC Level: 0 ppb
Rotational Position: 48 RPM
Input Pressure: 6 PSI
Temperature: 24°C
Answer:  failure<|endoftext|>


2322

In [10]:
# ============================================================================
# STEP 7: Prepare GRPO Dataset
# ============================================================================
grpo_df = machine_df[~machine_df.index.isin(pretrain_df.index)].copy()

# Balance the dataset
failure_samples = grpo_df[grpo_df['status'] == 'failure']
normal_samples = grpo_df[grpo_df['status'] == 'normal']

n_failure = len(failure_samples)
n_normal = len(normal_samples)

if n_failure < n_normal:
    normal_samples = normal_samples.sample(n=n_failure, random_state=42)
else:
    failure_samples = failure_samples.sample(n=n_normal, random_state=42)

grpo_df = pd.concat([failure_samples, normal_samples]).reset_index(drop=True)
grpo_df = grpo_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\nGRPO dataset class distribution:")
print(grpo_df['status'].value_counts())

def format_grpo_dataset(row):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""

    return {
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        "answer": row['status']
    }

grpo_dataset = grpo_df.apply(format_grpo_dataset, axis=1).tolist()
grpo_dataset = pd.DataFrame(grpo_dataset)
grpo_dataset = Dataset.from_pandas(grpo_dataset)

print(f"\nGRPO dataset size: {len(grpo_dataset)}")


GRPO dataset class distribution:
status
normal     386
failure    386
Name: count, dtype: int64

GRPO dataset size: 772


In [11]:
# ============================================================================
# STEP 8: Simplified Reward Functions (NO REASONING TAGS)
# ============================================================================
import torch.nn.functional as F

def _build_prompt_ids_from_row(row, tokenizer):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""

    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    return tokenizer(text, return_tensors="pt").to("cuda")

def _sequence_logprob(model, tokenizer, prompt_inputs, candidate_text):
    cand_ids = tokenizer(candidate_text, add_special_tokens=False, return_tensors="pt").to("cuda").input_ids
    prompt_ids = prompt_inputs["input_ids"]
    attn = prompt_inputs["attention_mask"]
    full_ids = torch.cat([prompt_ids, cand_ids], dim=1)
    full_attn = torch.cat([attn, torch.ones_like(cand_ids)], dim=1)

    with torch.no_grad():
        out = model(input_ids=full_ids, attention_mask=full_attn)
        logits = out.logits[:, -cand_ids.shape[1]-1:-1, :]
        logprobs = F.log_softmax(logits, dim=-1)
        gathered = logprobs.gather(-1, cand_ids.unsqueeze(-1)).squeeze(-1)
        return gathered.sum().item()

def predict_with_logprob(row, model, tokenizer):
    """Primary inference: score two labels and pick argmax."""
    prompt_inputs = _build_prompt_ids_from_row(row, tokenizer)
    lp_fail = _sequence_logprob(model, tokenizer, prompt_inputs, "failure")
    lp_norm = _sequence_logprob(model, tokenizer, prompt_inputs, "normal")
    return 'failure' if lp_fail > lp_norm else 'normal'

def check_answer(prompts, completions, answer, **kwargs):
    """Reward correct predictions with balanced class weighting"""
    responses = [completion[0]["content"].strip().lower() for completion in completions]

    scores = []
    for guess, true_answer in zip(responses, answer):
        true_answer = true_answer.strip().lower()

        # Simple exact match check
        if guess == true_answer:
            # Balanced rewards for both classes
            scores.append(5.0)
        elif true_answer in guess or guess in true_answer:
            scores.append(2.0)
        else:
            # Balanced penalties to avoid bias
            scores.append(-5.0)

    return scores

def diversity_reward(prompts, completions, answer, **kwargs):
    """Encourage diverse predictions to prevent single-class bias"""
    responses = [completion[0]["content"].strip().lower() for completion in completions]

    failure_count = sum(1 for r in responses if 'failure' in r)
    normal_count = sum(1 for r in responses if 'normal' in r)
    total_valid = failure_count + normal_count

    scores = []
    if total_valid > 0:
        ratio = min(failure_count, normal_count) / max(failure_count, normal_count, 1)
        # Reward balanced predictions
        diversity_score = ratio * 2.0  # Max 2.0 when perfectly balanced
        scores = [diversity_score] * len(completions)
    else:
        scores = [0] * len(completions)

    return scores

def length_penalty(prompts, completions, answer, **kwargs):
    """Penalize overly long responses (should be just one word)"""
    responses = [completion[0]["content"].strip() for completion in completions]

    scores = []
    for response in responses:
        words = response.split()
        if len(words) == 1:
            scores.append(1.0)  # Perfect length
        elif len(words) <= 3:
            scores.append(0)  # Acceptable
        else:
            scores.append(-2.0 * len(words))  # Penalty for verbosity

    return scores

global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def print_examples(prompts, completions, answer, **kwargs):
    """Print examples periodically"""
    global PRINTED_TIMES, PRINT_EVERY_STEPS

    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        question = prompts[0][-1]["content"]
        response = completions[0][0]["content"]

        all_predictions = [c[0]["content"].strip().lower() for c in completions]
        failure_count = sum(1 for p in all_predictions if 'failure' in p)
        normal_count = sum(1 for p in all_predictions if 'normal' in p)

        print("\n" + "="*60)
        print(f"STEP {PRINTED_TIMES}")
        print("="*60)
        print(f"Batch Prediction Distribution: Normal={normal_count}, Failure={failure_count}")
        print(f"\nTrue Status: {answer[0]}")
        print(f"Model Response: {response}")
        print("="*60)

    PRINTED_TIMES += 1
    return [0] * len(completions)



In [12]:
# ============================================================================
# STEP 9: GRPO Training (Simplified)
# ============================================================================
from vllm import SamplingParams
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256
max_completion_length = 50  # Just need one word

vllm_sampling_params = SamplingParams(
    min_p=0.05,
    top_p=0.9,
    top_k=50,
    temperature=0.8,  # Lower temperature for more deterministic outputs
    seed=3407,
    stop=[tokenizer.eos_token],
    include_stop_str_in_output=True,
)

training_args = GRPOConfig(
    vllm_sampling_params=vllm_sampling_params,
    temperature=0.8,
    learning_rate=5e-6,  # Slightly higher for simpler task
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_generations=4,  # Fewer generations needed for simple task
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    num_train_epochs=3,
    save_steps=50,
    output_dir="outputs_no_reasoning",
    report_to="none",
)

grpo_trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=grpo_dataset,
    reward_funcs=[
        check_answer,
        diversity_reward,
        length_penalty,
        print_examples,
    ],
)

print("\n" + "="*50)
print("Starting GRPO training (no reasoning)...")
print("="*50)
grpo_trainer.train()


Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 2 to the `num_generations` of 4

Starting GRPO training (no reasoning)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 772 | Num Epochs = 3 | Total steps = 1,158
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


WARNING 10-10 07:18:03 [processor.py:215] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.

STEP 0
Batch Prediction Distribution: Normal=2, Failure=4

True Status: normal
Model Response:  failure


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / check_answer / mean,rewards / check_answer / std,rewards / diversity_reward / mean,rewards / diversity_reward / std,rewards / length_penalty / mean,rewards / length_penalty / std,rewards / print_examples / mean,rewards / print_examples / std
1,0.000700,0.750000,5.386751,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.326144,-1.250000,5.175492,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2,0.000200,-1.166667,2.886751,2.125000,2.000000,3.000000,0.000000,2.125000,2.000000,3.000000,0.120245,-2.500000,4.629100,0.333333,0.000000,1.000000,0.000000,0.000000,0.000000
3,0.000800,0.150000,5.386751,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.378140,-1.250000,5.175492,0.400000,0.000000,1.000000,0.000000,0.000000,0.000000
4,0.000700,-8.541667,14.750001,8.000000,2.000000,50.000000,0.125000,2.000000,2.000000,2.000000,0.351581,-3.750000,3.535534,0.333333,0.000000,-5.125000,17.324116,0.000000,0.000000
5,0.000600,1.250000,5.386751,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.279894,-1.250000,5.175492,1.500000,0.000000,1.000000,0.000000,0.000000,0.000000
6,0.001200,-2.475000,2.750000,2.125000,2.000000,3.000000,0.000000,2.125000,2.000000,3.000000,0.578989,-3.750000,3.535534,0.400000,0.000000,0.875000,0.353553,0.000000,0.000000
7,0.000900,-1.750000,2.500000,2.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.440464,-3.750000,3.535534,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
8,0.001500,1.083333,5.386751,2.125000,2.000000,3.000000,0.000000,2.125000,2.000000,3.000000,0.727283,-1.250000,5.175492,1.333333,0.000000,1.000000,0.000000,0.000000,0.000000
9,0.000700,-7.791667,15.250000,8.000000,2.000000,50.000000,0.125000,2.000000,2.000000,2.000000,0.350537,-3.750000,3.535534,1.333333,0.000000,-5.375000,18.031223,0.000000,0.000000
10,0.000900,2.250000,5.386751,2.250000,2.000000,3.000000,0.000000,2.250000,2.000000,3.000000,0.443635,1.250000,5.175492,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000



STEP 5
Batch Prediction Distribution: Normal=1, Failure=5

True Status: normal
Model Response:  failure

STEP 10
Batch Prediction Distribution: Normal=1, Failure=6

True Status: failure
Model Response:  normal

STEP 15
Batch Prediction Distribution: Normal=1, Failure=5

True Status: normal
Model Response: 20

STEP 20
Batch Prediction Distribution: Normal=3, Failure=5

True Status: normal
Model Response:  failure

STEP 25
Batch Prediction Distribution: Normal=5, Failure=3

True Status: normal
Model Response:  failure

STEP 30
Batch Prediction Distribution: Normal=2, Failure=6

True Status: normal
Model Response:  failure

STEP 35
Batch Prediction Distribution: Normal=2, Failure=6

True Status: normal
Model Response:  failure

STEP 40
Batch Prediction Distribution: Normal=1, Failure=7

True Status: failure
Model Response:  failure

STEP 45
Batch Prediction Distribution: Normal=6, Failure=2

True Status: normal
Model Response:  normal

STEP 50
Batch Prediction Distribution: Normal=3, Fai

TrainOutput(global_step=1158, training_loss=0.001474247078314972, metrics={'train_runtime': 5668.4961, 'train_samples_per_second': 0.409, 'train_steps_per_second': 0.204, 'total_flos': 0.0, 'train_loss': 0.001474247078314972})

In [13]:
# ============================================================================
# STEP 10: Test the Model
# ============================================================================
print("\n" + "="*50)
print("Testing final model (no reasoning):")
print("="*50)

test_df = machine_df[~machine_df.index.isin(pretrain_df.index) &
                      ~machine_df.index.isin(grpo_df.index)].head(3)

if len(test_df) == 0:
    test_df = grpo_df.sample(n=3, random_state=123)

for i, (idx, row) in enumerate(test_df.iterrows()):
    problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""

    text = tokenizer.apply_chat_template(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": problem},
        ],
        tokenize=False,
        add_generation_prompt=True,
    )

    print(f"\n{'='*60}")
    print(f"Test Example {i+1} (Actual: {row['status']})")
    print('='*60)
    _ = model.generate(
        **tokenizer(text, return_tensors="pt").to("cuda"),
        temperature=0.1,  # Low temperature for deterministic prediction
        max_new_tokens=50,
        streamer=TextStreamer(tokenizer, skip_prompt=True),
    )



Testing final model (no reasoning):

Test Example 1 (Actual: normal)
 normal<|endoftext|>

Test Example 2 (Actual: normal)
 failure<|endoftext|>

Test Example 3 (Actual: normal)
 failure<|endoftext|>


In [14]:
# ============================================================================
# STEP 11: Evaluate Model on All Data
# ============================================================================
print("\n" + "="*50)
print("Evaluating model on entire dataset (no reasoning)...")
print("="*50)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time

all_data_df = machine_df.copy()

def predict_machine_status(row, model, tokenizer):
    """Make a prediction using log-prob scoring."""
    try:
        return predict_with_logprob(row, model, tokenizer)
    except Exception:
        # Fallback to generation
        problem = f"""Predict machine failure based on sensor data:
Footfall: {row['footfall']}
Temperature Mode: {row['tempmode']}
Air Quality: {row['aq']} AQI
Ultrasonic Sensor: {row['uss']}
Current Sensor: {row['cs']} A
VOC Level: {row['voc']} ppb
Rotational Position: {row['rp']} RPM
Input Pressure: {row['ip']} PSI
Temperature: {row['temperature']}°C"""

        text = tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": problem},
            ],
            tokenize=False,
            add_generation_prompt=True,
        )

        outputs = model.generate(
            **tokenizer(text, return_tensors="pt").to("cuda"),
            temperature=0.1,
            max_new_tokens=50,
            do_sample=False,
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = response.split("Answer:")[-1].strip().lower()

        if 'failure' in response:
            return 'failure'
        elif 'normal' in response:
            return 'normal'
        else:
            return 'normal'  # Conservative default

print(f"\nMaking predictions on {len(all_data_df)} samples...")
print("This may take a few minutes...\n")

predictions = []
true_labels = []
start_time = time.time()

for idx, row in all_data_df.iterrows():
    if idx % 10 == 0:
        print(f"Progress: {idx}/{len(all_data_df)} samples processed...")

    pred = predict_machine_status(row, model, tokenizer)
    predictions.append(pred)
    true_labels.append(row['status'])

end_time = time.time()
print(f"\n✓ Completed {len(all_data_df)} predictions in {end_time - start_time:.2f} seconds")
print(f"  Average time per prediction: {(end_time - start_time) / len(all_data_df):.3f} seconds")

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, pos_label='failure', zero_division=0)
recall = recall_score(true_labels, predictions, pos_label='failure', zero_division=0)
f1 = f1_score(true_labels, predictions, pos_label='failure', zero_division=0)

cm = confusion_matrix(true_labels, predictions, labels=['normal', 'failure'])

print("\n" + "="*60)
print("EVALUATION RESULTS (NO REASONING)")
print("="*60)
print(f"\nOverall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision (Failure): {precision:.4f}")
print(f"Recall (Failure): {recall:.4f}")
print(f"F1-Score (Failure): {f1:.4f}")

print("\nConfusion Matrix:")
print("                 Predicted")
print("                Normal  Failure")
print(f"Actual Normal    {cm[0][0]:6d}  {cm[0][1]:7d}")
print(f"       Failure   {cm[1][0]:6d}  {cm[1][1]:7d}")

print("\nDetailed Classification Report:")
print(classification_report(true_labels, predictions, target_names=['normal', 'failure']))

# Class-wise accuracy
normal_correct = cm[0][0]
normal_total = cm[0][0] + cm[0][1]
failure_correct = cm[1][1]
failure_total = cm[1][0] + cm[1][1]

print(f"\nClass-wise Performance:")
print(f"  Normal class accuracy:  {normal_correct}/{normal_total} ({normal_correct/normal_total*100:.2f}%)")
print(f"  Failure class accuracy: {failure_correct}/{failure_total} ({failure_correct/failure_total*100:.2f}%)")

# Save results
results_df = pd.DataFrame({
    'true_label': true_labels,
    'predicted_label': predictions,
    'correct': [t == p for t, p in zip(true_labels, predictions)]
})

results_df = pd.concat([all_data_df.reset_index(drop=True), results_df], axis=1)
results_df.to_csv('model_evaluation_results_no_reasoning.csv', index=False)
print(f"\n✓ Detailed results saved to 'model_evaluation_results_no_reasoning.csv'")

print("\n" + "="*60)
print("COMPARISON: REASONING VS NO REASONING")
print("="*60)
print("\nWITH REASONING:")
print("  Overall Accuracy: 41.63%")
print("  Precision: 0.4163")
print("  Recall: 1.0000")
print("  F1-Score: 0.5879")
print("\nNO REASONING:")
print(f"  Overall Accuracy: {accuracy*100:.2f}%")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")
print("="*60)



Evaluating model on entire dataset (no reasoning)...

Making predictions on 944 samples...
This may take a few minutes...

Progress: 0/944 samples processed...
Progress: 10/944 samples processed...
Progress: 20/944 samples processed...
Progress: 30/944 samples processed...
Progress: 40/944 samples processed...
Progress: 50/944 samples processed...
Progress: 60/944 samples processed...
Progress: 70/944 samples processed...
Progress: 80/944 samples processed...
Progress: 90/944 samples processed...
Progress: 100/944 samples processed...
Progress: 110/944 samples processed...
Progress: 120/944 samples processed...
Progress: 130/944 samples processed...
Progress: 140/944 samples processed...
Progress: 150/944 samples processed...
Progress: 160/944 samples processed...
Progress: 170/944 samples processed...
Progress: 180/944 samples processed...
Progress: 190/944 samples processed...
Progress: 200/944 samples processed...
Progress: 210/944 samples processed...
Progress: 220/944 samples pro